## Using pytorch module 

In [13]:
import numpy as np
import torch
import torch.nn as nn

In [14]:
# mse = MSE(prediction, targets)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [15]:
from torch.utils.data import TensorDataset

In [16]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [17]:
from torch.utils.data import DataLoader

In [18]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[ 22.,  37.],
        [ 56.,  70.],
        [ 81., 101.],
        [103., 119.],
        [119., 133.]])


In [23]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3912, -0.4723,  0.2515],
        [ 0.5328, -0.4992, -0.0968]], requires_grad=True)
Parameter containing:
tensor([-0.3397,  0.2201], requires_grad=True)


In [22]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2744, -0.1774, -0.5177],
         [ 0.0438,  0.5407,  0.4472]], requires_grad=True),
 Parameter containing:
 tensor([0.2786, 0.0705], requires_grad=True)]

In [24]:
preds = model(inputs)
preds

tensor([[-49.7270,   1.5081],
        [-61.4059,  -1.4171],
        [-83.0769, -25.9308],
        [-51.2438,  29.5223],
        [-55.0701, -17.7142]], grad_fn=<AddmmBackward0>)

## Before optimiser

In [26]:
import torch.nn.functional as F
loss_fn = F.mse_loss

In [27]:
loss = loss_fn(model(inputs), targets)
loss

tensor(16182.9609, grad_fn=<MseLossBackward0>)

#  After optimiser

In [28]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [32]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    #do fiting  for num times
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            #Reset the gradients to zero
            opt.zero_grad()
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))   

In [33]:
fit(100, model, loss_fn, opt, train_dl)



Epoch [10/100], Loss: 829.2302
Epoch [20/100], Loss: 724.9564
Epoch [30/100], Loss: 640.1721
Epoch [40/100], Loss: 565.6091
Epoch [50/100], Loss: 499.9140
Epoch [60/100], Loss: 442.0265
Epoch [70/100], Loss: 391.0155
Epoch [80/100], Loss: 346.0608
Epoch [90/100], Loss: 306.4404
Epoch [100/100], Loss: 271.5183


In [34]:
print(model(inputs))

tensor([[ 60.5851,  76.8916],
        [ 85.4223, 101.4541],
        [105.8811, 120.5938],
        [ 41.3754,  74.6890],
        [ 95.6054,  98.5605]], grad_fn=<AddmmBackward0>)


In [35]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [37]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[59.0249, 76.0491]], grad_fn=<AddmmBackward0>)